In [1]:
from random import random
from queue import Queue
import time
import numpy as np
from prevo.misc import PeriodicThreadedSystem
from prevo.plot import OscilloGraph, NumericalGraph
import oclock
%matplotlib qt5

# Dummy sensors

## Time data as individual points

In [2]:
class ManipBase(PeriodicThreadedSystem):
    
    def __init__(self, interval=0.1):
        super().__init__(interval=interval, precise=False)
        self.queue = Queue()
        
    def get_values(self):
        """Define in subclasses"""
        pass
        
    def _update(self):
        with oclock.measure_time() as data:
            values = self._get_values()
        data['values'] = values
        data['name'] = self.name
        self.queue.put(data)

In [3]:
class Manip1(ManipBase):
    
    name = 'A'
    data_types = 'P (Pa)', 'T (°C)', 'P (Pa)'
    
    def _get_values(self):
        val1 = random()
        val2 = 25 + 2 * random()
        val3 = random()
        return val1, val2, val3

In [4]:
class Manip2(ManipBase):
    
    name = 'B'
    data_types = 'T (°C)', 'valve'
    
    def _get_values(self):
        val1 = 20 + random()
        val2 = int(random() * 100)
        return val1, val2

## Time data as arrays

In [5]:
class ManipArray(PeriodicThreadedSystem):
    
    name = 'C'
    data_types = 'P (Pa)', 'P (Pa)'
    
    def __init__(self, interval=1, npts=100):
        super().__init__(interval=interval, precise=False)
        self.queue = Queue()
        self.interval = interval
        self.npts = npts
        
    def _update(self):
        data = {}
        t0 = time.time() - self.interval
        data['time (unix)'] = t0 + np.linspace(start=0,
                                               stop=self.interval,
                                               num=self.npts)
        data_array_a = 0.1 * np.random.rand(self.npts) + 0.7
        data_array_b = 0.2 * np.random.rand(self.npts) + 0.3
        
        data['values'] = data_array_a, data_array_b
        data['name'] = self.name
        
        self.queue.put(data)

## Misc. sensor config

In [6]:
data_types = {'A': Manip1.data_types,
              'B': Manip2.data_types,
              'C': ManipArray.data_types}

data_ranges = {'P (Pa)': (0, 1),
               'T (°C)': (20, 30),
               'valve': (0, 100)}

In [7]:
manip1 = Manip1(interval=0.02)
manip1.start()

manip2 = Manip2(interval=0.7)
manip2.start()

manip3 = ManipArray(interval=1)
manip3.start()

In [8]:
colors = {'fig': 'lightgrey',  # set color of figure background around axes
          'B': ('k', 'b')}      # set color of curves of the two channels of sensor B

# Regular numerical plot

### Data as single time values

In [9]:
numplot = NumericalGraph(names=('A', 'B'),
                         data_types=data_types,
                         linestyle='.')

numplot.run(q_plot={'A': manip1.queue,
                    'B': manip2.queue},
            dt_graph=0.2)

### Data as arrays

In [ ]:
numplot = NumericalGraph(names=('C',),
                         data_types=data_types,
                         linestyle='.',
                         data_as_array=True,
                         time_conversion='numpy')

numplot.run(q_plot={'C': manip3.queue}, dt_graph=0.2)

### Mix of single times and arrays

In [ ]:
numplot = NumericalGraph(names=('A', 'B', 'C'),
                         data_types=data_types,
                         linestyle='.',
                         data_as_array={'A': False, 'B': False, 'C': True},
                         time_conversion='pandas')

numplot.run(q_plot={'A': manip1.queue, 'B': manip2.queue, 'C': manip3.queue}, dt_graph=0.2)

# Oscilloscope-like graph

### Data as single time values

In [ ]:
oscillo = OscilloGraph(names=('A', 'B'),
                       data_types=data_types,
                       data_ranges=data_ranges,
                       window_width=10,
                       colors=colors,
                       linestyle='.-')

oscillo.run(q_plot={'A': manip1.queue, 'B': manip2.queue}, dt_graph=0.02)

### Data as arrays

In [ ]:
oscillo = OscilloGraph(names=('C',),
                       data_types=data_types,
                       data_ranges=data_ranges,
                       window_width=5.7,
                       colors=colors,
                       linestyle='.',
                       data_as_array=True)

oscillo.run(q_plot={'C': manip3.queue}, dt_graph=0.02)

### Mix of single times and arrays

In [ ]:
oscillo = OscilloGraph(names=('A', 'B', 'C'),
                       data_types=data_types,
                       data_ranges=data_ranges,
                       window_width=5.7,
                       colors=colors,
                       data_as_array={'A': False, 'B': False, 'C': True})

oscillo.run(q_plot={'A': manip1.queue, 'B': manip2.queue, 'C': manip3.queue}, dt_graph=0.02)

# Stop dummy sensors

In [ ]:
manip1.stop()
manip2.stop()
manip3.stop()